In [ ]:
import numpy as np
from skimage import io
from sklearn.feature_extraction import image
from scipy.stats import entropy
from sklearn.preprocessing import StandardScaler
import spectral

# Provide the path to the header file (.hdr) of your ENVI file
hdr_file_path = '/content/LR_234_bands.hdr'

# Load the ENVI file
image_data = spectral.open_image(hdr_file_path)

# Check the shape of the original image
original_shape = image_data.shape
print(f"Original Image Shape: {original_shape}")

# Make lines and samples equal by zero-padding
max_dimension = max(original_shape[0], original_shape[1])
padded_image = np.zeros((max_dimension, max_dimension, original_shape[2]), dtype=image_data.dtype)
padded_image[:original_shape[0], :original_shape[1], :] = image_data

# Check the shape of the padded image
padded_shape = padded_image.shape
print(f"Padded Image Shape: {padded_shape}")

# Create patches with equal lines and samples
patch_size = 30  # Adjust the patch size as needed
patches = image.extract_patches_2d(padded_image, (patch_size, patch_size, original_shape[2]))

# Reshape patches to have equal lines and samples
patches = patches.reshape(-1, patch_size, patch_size, original_shape[2])

# Calculate covariance matrix and spatial coherence for each band
num_patches = patches.shape[0]
covariance_matrices = []
spatial_coherence = []

for band in range(original_shape[2]):
    band_patches = patches[:, :, :, band].reshape(num_patches, -1)

    # Calculate covariance matrix
    covariance_matrix = np.cov(band_patches, rowvar=False)
    covariance_matrices.append(covariance_matrix)

    # Standardize data for spatial coherence
    scaler = StandardScaler()
    standardized_data = scaler.fit_transform(band_patches)

    # Calculate spatial coherence using entropy
    band_spatial_coherence = np.apply_along_axis(entropy, 1, standardized_data)
    spatial_coherence.append(band_spatial_coherence)

# Convert lists to arrays
covariance_matrices = np.array(covariance_matrices)
spatial_coherence = np.array(spatial_coherence)

# Calculate Renyi's entropy threshold for each band
thresholds = []
alpha = 2  # Adjust the alpha parameter as needed

for band in range(original_shape[2]):
    band_threshold = np.percentile(spatial_coherence[band], 100 - alpha)
    thresholds.append(band_threshold)

# Display the calculated thresholds for each band
for band, threshold in enumerate(thresholds):
    print(f"Band {band + 1}: Threshold = {threshold}")
